---
## <font color="red">1 | SYSTÈME DE</font> RAISONNEMENT
---

Dans ce notebook, vous apprendrez à implémenter un système de raisonnement avec des connaissances de base sur le monde des meurtres.

### <font color="red">1.1 - Les</font> paquets

In [1]:
from aima.logic import *
import nltk
from gtts import gTTS
import os
import speech_recognition as sr
kb = FolKB()
from aima.utils import expr
from nltk import load_parser
import sys

### <font color="red">1.2 - Le moteur d'</font> inférence

In [2]:
# Permet d'inferer qui est le meurtrier, quand, comment, où il a tué.
class CrimeInference:

    def __init__(self):
        self.weapons = ["Corde", "Fusil", "Couteau", "Clé_anglaise", "Matraque", "Poison"]
        self.rooms = ["Phare", "Radio_room", "Embarcadère", "Plage", "Observatoire", "Serre", "Générateur"]
        self.persons = ["Gouverneur_White", "Botaniste_Green", "Docteur_Brown", "Capitaine_Gray", "Ingenieur_Blue", "Gardien_de_phare_Red", "Professeur_Yellow"]
        
        # Liste de clauses (faits) qui seront stockées dans la base de connaissance.
        self.clauses = []        
        
        self.base_clauses()
        self.initialize_KB()
        self.inference_rules()
        
        # Base de connaissances (First-order logic - FOL)
        self.crime_kb = FolKB(self.clauses)

    # Déclaration dans la logique du premier ordre
    def base_clauses(self):
        # Le paramètre est une arme
        self.arme_clause = 'Arme({})'
        
        # Le paramètre est une pièce
        self.piece_clause = 'Piece({})'
        
        # Le paramètre est une persone
        self.personne_clause = 'Personne({})'

        # paramètre 1 : arme; paramètre 2 : pièce
        # p.ex.: Le couteau se trouve dans la cuisine
        self.weapon_room_clause = 'ArmePiece({},{})'

        # paramètre 1 : personne; paramètre 2 : pièce; paramètre 3 : heure
        # p.ex.: Botaniste_Green était dans la cuisine à 11h00
        self.person_room_hour_clause = 'Personne_Piece_Heure({}, {}, {})'

        # paramètre 1 : personne; paramètre 2 : piece
        # p.ex.: Botaniste_Green se trouve dans la cuisine
        self.person_room_clause = 'Personne_Piece({}, {})'

        # paramète 1 : personne
        # p. ex.: Botaniste_Green est mort
        self.dead_clause = 'EstMort({})'
        
        # paramète 1 : personne
        # p. ex.: Botaniste_Green est vivant
        self.alive_clause = 'EstVivant({})'

        # paramètre 1 : personne
        # p. ex.: Botaniste_Green est la victime
        self.victim_clause = 'Victime({})'

        # paramètre 1 : personne
        # p. ex.: Botaniste_Green a des marques au cou
        self.body_mark_clause = 'MarqueCou({})'

        # paramètre 1 : piece; paramètre 2 : piece
        self.room_different_clause = 'PieceDifferente({},{})'

        # paramètre 1 : piece; paramètre 2 : piece
        self.weapon_different_clause = 'ArmeDifferente({},{})'

        # paramètre 1 : heure
        self.crime_hour_clause = 'HeureCrime({})'

        # paramètre 1 : heure
        self.crime_hour_plus_one_clause = 'UneHeureApresCrime({})'

    def initialize_KB(self):
        # Clause pour differencier les pièces
        for i in range(len(self.rooms)):
            for j in range(len(self.rooms)):
                if i != j:
                    # Le phare est different de la cuisine = PieceDifferente(phare, Cuisine)
                    self.clauses.append(expr(self.room_different_clause.format(self.rooms[i], self.rooms[j])))

        # Clause pour differencier les armes
        for i in range(len(self.weapons)):
            for j in range(len(self.weapons)):
                if i != j:
                    # Le couteau est different de la corde = ArmeDifferente(Couteau, Corde)
                    self.clauses.append(expr(self.weapon_different_clause.format(self.weapons[i], self.weapons[j])))

        # Initialiser KB sur Armes, Pieces, Personnes
        for weapon in self.weapons:
            # Le couteau est une arme = Arme(Couteau)
            self.clauses.append(expr(self.arme_clause.format(weapon)))

        for room in self.rooms:
            # La cuisine est une pièce = Piece(Cuisine)
            self.clauses.append(expr(self.piece_clause.format(room)))

        for person in self.persons:
            # Mustar est une personne = Personne(Botaniste_Green)
            self.clauses.append(expr(self.personne_clause.format(person)))
    
    # Expressions dans la logique du premier ordre permettant de déduire les caractéristiques du meurtre
    def inference_rules(self):
        # Determine la piece du crime
        self.clauses.append(expr('EstMort(x) & Personne_Piece(x, y) ==> PieceCrime(y)'))

        # Determiner l'arme du crime
        self.clauses.append(expr('PieceCrime(x) & Arme(y) & Piece_Arme(y, x) ==> ArmeCrime(y)'))
        self.clauses.append(expr("EstMort(x) & MarqueCou(x) ==> ArmeCrime(Corde)"))
        self.clauses.append(expr("EstMort(x) & MarqueFusil(x) ==> ArmeCrime(Fusil)"))
        self.clauses.append(expr("EstMort(x) & MarqueCouteau(x) ==> ArmeCrime(Couteau)"))
        self.clauses.append(expr("EstMort(x) & MarqueCleAnglaise(x) ==> ArmeCrime(Clé_anglaise)"))
        self.clauses.append(expr("EstMort(x) & MarqueMatraque(x) ==> ArmeCrime(Matraque)"))
        self.clauses.append(expr("EstMort(x) & MarquePoison(x) ==> ArmeCrime(Poison)"))

        # Ajout dans CrimeInference.inference_rules()
        self.clauses.append(expr("Personne_Piece_Heure(p, r, h) & PieceCrime(r) & HeureCrime(h) ==> Suspect(p)"))
        self.clauses.append(expr("Personne_Piece_Heure(x, Phare, 14) ==> Possede(x, Corde)"))
        self.clauses.append(expr("(Possede(x, Corde) & ArmeCrime(Corde)) ==> SuspectArme(x, Corde)"))
        self.clauses.append(expr("(Suspect(x) & Possede(x, a) & ArmeCrime(a)) ==> Coupable(x)"))


        # Si la personne est morte alors elle est la victime et ce n'est pas un suicide
        self.clauses.append(expr('EstMort(x) ==> Victime(x)'))

        # Si la personne est morte alors elle est innocente et ce n'est pas un suicide
        self.clauses.append(expr('EstMort(x) ==> Innocent(x)'))

        # Si la personne est vivante et était dans une pièce
        # qui ne contient pas l'arme du crime, alors elle est innocente
        self.clauses.append(expr(
            'EstVivant(p) & UneHeureApresCrime(h1) & Personne_Piece_Heure(p,r2,h1) & PieceCrime(r1)'
            ' & PieceDifferente(r1,r2) & ArmeCrime(a1) & ArmePiece(a2,r2) & ArmeDifferente(a1,a2) ==> Innocent(p)'))

        # Si la personne se trouvait dans une piece qui contient l'arme
        # qui a tué la victime une heure après le meurtre alors elle est suspecte
        self.clauses.append(expr(
            'EstVivant(p) & UneHeureApresCrime(h1) & Personne_Piece_Heure(p,r2,h1) & PieceCrime(r1)'
            ' & PieceDifferente(r1,r2) & ArmeCrime(a) & ArmePiece(a,r2) ==> Suspect(p)'))

    # Ajouter des clauses, c'est-à-dire des faits, à la base de connaissances
    def add_clause(self, clause_string):
        self.crime_kb.tell(expr(clause_string))

    # Demander à la base de connaissances qui est la victime
    def get_victim(self):
        result = self.crime_kb.ask(expr('Victime(x)'))
        if not result:
            return False
        else:
            return result[x]
        
    # Demander à la base de connaissances la pièce du meurtre
    def get_crime_room(self):
        result = self.crime_kb.ask(expr('PieceCrime(x)'))
        if not result:
            return False
        else:
            return result[x]

    # Demander à la base de connaissances l'arme du meurtrier
    def get_crime_weapon(self):
        result = self.crime_kb.ask(expr('ArmeCrime(x)'))
        if not result:
            return result
        else:
            return result[x]

    # Demander à la base de connaissances l'heure du meurtre
    def get_crime_hour(self):
        result = self.crime_kb.ask(expr('HeureCrime(x)'))
        if not result:
            return result
        else:
            return result[x]

    def get_crime_hour_plus_one(self):
        result = self.crime_kb.ask(expr('UneHeureApresCrime(x)'))
        if not result:
            return result
        else:
            return result[x]
    
    # Demander à la base de connaissances le suspect
    def get_suspect(self):
        result = self.crime_kb.ask(expr('Suspect(x)'))
        if not result:
            return result
        else:
            return result[x]

    # Demander à la base de connaissances la liste d'innocents
    def get_innocent(self):
        result = list(fol_bc_ask(self.crime_kb, expr('Innocent(x)')))
        res = []

        for elt in result:
            if not res.__contains__(elt[x]):
                res.append(elt[x])
        return res

### <font color="red">1.3 - Les faits et </font> la déduction
---
Maintenant que nous avons le moteur de déduction, il est temps de commencer à alimenter la base de connaissances avec les faits du monde.

Comme nous communiquerons les faits au moteur d’inférence en français, plusieurs grammaires ont été créées pour interpréter les phrases. Celles-ci se trouvent dans le dossier `grammars`.

À partir des grammaires, nous pouvons analyser les phrases et puis les transformer en expressions logiques du premier ordre. C'est à cela que servent les fonctions `results_as_string` et `to_fol`. 

In [3]:
# Cette fonction retourne le format d'une expression logique de premier ordre
def results_as_string(results):
    res = ''
    for result in results:
        # synrep = syntactic representation
        # semrep = semantic representation
        for (synrep, semrep) in result:            
            res += str(semrep)
    return res

La fonction `nltk.interpret_sents` donne une représentation de premier ordre d'un fait en fraçais à partir d'une grammaire. La représentation est récupérée et transmise à la fonction `results_as_string` pour la formater sous la forme Exp(x). Enfin, l'expression mise en forme est renvoyée pour l'ajouter à la base de connaissances.

In [4]:
# Cette fonction transforme une phrase en fraçais dans une expression logique du premier ordre
def to_fol(fact, grammar):
    sent = results_as_string(nltk.interpret_sents(fact, grammar))
    print(sent)
    return sent    

# 2. Reconnaissance vocale

In [ ]:
def reconnaitre_parole():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Parlez maintenant...")
        r.adjust_for_ambient_noise(source)  # réduit le bruit ambiant
        audio = r.listen(source)

    try:
        texte = r.recognize_google(audio, language="fr-FR")
        print("Vous avez dit : " + texte)
    except sr.UnknownValueError:
        print("Je n'ai pas compris.")
    except sr.RequestError as e:
        print("Erreur avec le service Google : {0}".format(e))

# Test
# reconnaitre_parole()


In [ ]:
import pygame
import time

def speak_french_gtts(text):
    try:
        tts = gTTS(text=text, lang='fr')
        tts.save("speech.mp3")
        pygame.mixer.init()
        pygame.mixer.music.load("speech.mp3")
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            time.sleep(0.1)
        pygame.mixer.quit()
        os.remove("speech.mp3")
    except Exception as e:
        print("Erreur synthèse vocale:", e)

# speak_french_gtts("Bienvenue dans le jeu de raisonnement criminel.")


pygame 2.6.1 (SDL 2.28.4, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


## 2 | FONCTIONS INTERACTIONS TEXTE EN FRANÇAIS

In [ ]:
import os
import re
import sys

GRAMMAR_DIR = os.path.join(os.getcwd(), "grammars")

def choisir_grammaire(phrase):
    phrase = phrase.lower()

    if "mort à" in phrase or re.search(r"mort[e]? .*à \d{1,2}h", phrase):
        return "personne_morte_heure.fcfg"
    elif "mort" in phrase:
        return "personne_morte.fcfg"
    elif "marque" in phrase:
        return "personne_marque.fcfg"
    elif any(arme in phrase for arme in ["corde", "couteau", "fusil", "clé", "cle", "matraque", "poison"]):
        return "arme_piece.fcfg"
    elif re.search(r"(était|est|se trouve).*dans.*à \d{1,2}h", phrase):
        return "personne_piece_heure.fcfg"
    elif re.search(r"(était|est|se trouve).*dans", phrase):
        return "personne_piece.fcfg"
    else:
        return None
        

def lire_fait_francais(agent):
    phrase = input("→ Saisis une phrase (ex: Capitaine_Gray était dans le phare à 12h) :\n> ").strip()
    grammar_file = choisir_grammaire(phrase)
    if not grammar_file:
        print("Aucun fichier de grammaire trouvé pour cette phrase.")
        return

    grammar_path = os.path.join(GRAMMAR_DIR, grammar_file)
    tokens = phrase.split()

    try:
        parser = load_parser(f'file:{grammar_path}', trace=0)
        trees = list(parser.parse(tokens))
        sems = [tree.label()['SEM'] for tree in trees if 'SEM' in tree.label()]
        if sems:
            for sem in sems:
                print(f"Fait reconnu : {sem}")
                agent.add_clause(str(sem))
        else:
            print("Aucune sémantique détectée pour cette phrase.")
    except Exception as e:
        print(f"Erreur d'analyse avec {grammar_file} : {e}")

GRAMMAR_DIR = "./grammars"

def extraire_constantes_depuis_grammaire(fichier_fcfg, categorie=None):
    constantes = set()
    with open(fichier_fcfg, 'r', encoding='utf-8') as f:
        for ligne in f:
            if categorie and not ligne.strip().startswith(categorie + "["):
                continue

            match = re.search(r"SEM=<([^>]+)>", ligne)
            if match:
                valeur = match.group(1)

                # On ignore les sémantiques de fonction logique
                if "(" in valeur or "?" in valeur or "," in valeur:
                    continue

                # Nettoyage simple
                constantes.add(valeur.replace(" ", "_"))
    return sorted(list(constantes))



# Trop complexe pour l'énoncé mais je le laisse ici (On ne demande que 5 questions)
def poser_questions_dynamique(kb: FolKB):
    deja_posees = set()
    faits_confirms = {
        "arme": False,
        "victime": False,
        "heure": False,
    }

    print("\nL'agent tente de résoudre l'enquête...")
    speak_french_gtts("L'agent tente de résoudre l'enquête.")

    armes = extraire_constantes_depuis_grammaire("grammars/arme_piece.fcfg", "N")
    personnes = extraire_constantes_depuis_grammaire("grammars/personne_piece.fcfg", "Nprop")
    heures = extraire_constantes_depuis_grammaire("grammars/personne_piece_heure.fcfg", "Nh")
    lieux = extraire_constantes_depuis_grammaire("grammars/personne_piece_heure.fcfg", "Npiece")

    while True:
        coupables = list(kb.ask_generator(expr("Coupable(x)")))
        if coupables:
            print("\nCoupable trouvé :")
            for res in coupables:
                print("  -", res[expr("x")])
                speak_french_gtts(f"Le coupable est {res[expr('x')]}")
            break

        nouvelle_info = False

        # --- ARME ---
        if not faits_confirms["arme"]:
            for arme in armes:
                fait = f"ArmeCrime({arme})"
                if fait not in deja_posees:
                    question = f"L’arme du crime est-elle {arme} ?"
                    print("Question :", question)
                    speak_french_gtts(question)
                    reponse = reconnaitre_parole()
                    deja_posees.add(fait)
                    if reponse and "oui" in reponse.lower():
                        kb.tell(expr(fait))
                        faits_confirms["arme"] = True
                        nouvelle_info = True
                    break
            if nouvelle_info:
                continue

        # --- VICTIME ---
        if not faits_confirms["victime"]:
            for p in personnes:
                fait = f"EstMort({p})"
                if fait not in deja_posees:
                    question = f"La victime est-elle {p} ?"
                    print("Question :", question)
                    speak_french_gtts(question)
                    reponse = reconnaitre_parole()
                    deja_posees.add(fait)
                    if reponse and "oui" in reponse.lower():
                        kb.tell(expr(fait))
                        faits_confirms["victime"] = True
                        nouvelle_info = True
                    break
            if nouvelle_info:
                continue

        # --- HEURE ---
        if not faits_confirms["heure"]:
            for h in heures:
                h_clean = h.replace("h00", "").replace("h", "")
                fait = f"HeureCrime({h_clean})"
                if fait not in deja_posees:
                    question = f"Le crime a-t-il eu lieu à {h} ?"
                    print("Question :", question)
                    speak_french_gtts(question)
                    reponse = reconnaitre_parole()
                    deja_posees.add(fait)
                    if reponse and "oui" in reponse.lower():
                        kb.tell(expr(fait))
                        faits_confirms["heure"] = True
                        nouvelle_info = True
                    break
            if nouvelle_info:
                continue

        # --- PERSONNE - LIEU - HEURE ---
        for p in personnes:
            for l in lieux:
                for h in heures:
                    h_clean = h.replace("h00", "").replace("h", "")
                    fait = f"Personne_Piece_Heure({p},{l},{h_clean})"
                    if fait not in deja_posees:
                        question = f"{p} était-il dans le {l} à {h} ?"
                        print("Question :", question)
                        speak_french_gtts(question)
                        reponse = reconnaitre_parole()
                        deja_posees.add(fait)
                        if reponse and "oui" in reponse.lower():
                            kb.tell(expr(fait))
                            nouvelle_info = True
                        break
                if nouvelle_info:
                    break
            if nouvelle_info:
                break

        if not nouvelle_info:
            print("L'agent ne peut pas avancer sans nouvelles informations.")
            speak_french_gtts("Je ne peux pas continuer sans nouvelles informations.")
            break


    
def menu_general(agent):
    print("\nSouhaitez-vous :")
    print("1. Ajouter des faits manuellement")
    print("2. Poser des questions à l'agent (avec faits de base fournis)")
    print("3. Laisser l’agent poser des questions dynamiques")
    
    sous_choix = input("Choisissez (1, 2 ou 3) : ")

    if sous_choix == "1":
        # Mode ajout de faits
        while True:
            print("\n--- Mode : Ajout de faits ---")
            print("Tapez un fait ou 'retour' pour quitter.")
            phrase = input("> ").strip()
            if phrase.lower() == "retour":
                print("Fin du mode ajout de faits.")
                break
            else:
                lire_fait_francais(agent)

    elif sous_choix == "2":
        # Mode questions manuelles
        ajouter_faits_de_base(agent)
        while True:
            poser_question(agent)
            continuer = input("\nSouhaitez-vous poser une autre question ? (o/n) : ")
            if continuer.lower() != "o":
                print("Fin du mode questions.")
                break

    elif sous_choix == "3":
        # Mode questions dynamiques (agent qui interroge l'humain)
        print("\n--- Mode : Questions dynamiques par l'agent ---")
        ajouter_faits_de_base(agent)
        poser_questions_dynamique(kb)

    else:
        print("Choix invalide. Retour au menu principal annulé.")


            
def poser_question(agent):
    print("\n--- Questions possibles ---")
    print("1. Qui est la victime ?")
    print("2. Quelle est la pièce du crime ?")
    print("3. Quelle est l’arme du crime ?")
    print("4. À quelle heure le crime a-t-il eu lieu ?")
    print("5. Qui est suspect ?")
    print("6. Qui est innocent ?")
    print("7. Retour au menu")

    choix = input("Votre question (1 à 7) : ")

    if choix == "1":
        print("Victime :", agent.get_victim())
    elif choix == "2":
        print("Pièce du crime :", agent.get_crime_room())
    elif choix == "3":
        print("Arme du crime :", agent.get_crime_weapon())
    elif choix == "4":
        print("Heure du crime :", agent.get_crime_hour())
    elif choix == "5":
        print("Suspect :", agent.get_suspect())
    elif choix == "6":
        print("Innocents :", agent.get_innocent())
    elif choix == "7":
        return
    else:
        print("Choix invalide.")
        
def ajouter_faits_de_base(agent):
    print("Ajout des faits de base pour commencer l'enquête...\n")

    agent.add_clause("EstMort(Gouverneur_White)")
    agent.add_clause("MarqueCou(Gouverneur_White)")
    agent.add_clause("Personne_Piece(Gouverneur_White, Phare)")
    agent.add_clause("Personne_Piece(Docteur_Brown, Phare)")
    agent.add_clause("EstMortHeure(Gouverneur_White, 14)")
    agent.add_clause("Personne_Piece_Heure(Docteur_Brown, Phare, 14)")
    agent.add_clause("HeureCrime(14)")
    agent.add_clause("UneHeureApresCrime(15)")



Nous créons maintenant une instance du moteur d'inférence et commençons à ajouter les faits à la base de connaissances. Les faits sont communiqués en français, interprétés par la grammaire appropriée et ajoutés.

In [6]:
agent = CrimeInference()

In [ ]:
print("Bienvenue dans l'assistant d'enquête!")

menu_general(agent)

Bienvenue dans l'assistant d'enquête!
Souhaitez-vous utiliser l'interface textuelle ou vocale?
1. Textuel (saisie clavier)
2. Vocal (reconnaissance vocale)

Souhaitez-vous :
1. Ajouter des faits manuellement
2. Poser des questions à l'agent (avec faits de base fournis)
3. Laisser l’agent poser des questions dynamiques

--- Mode : Ajout de faits ---
Tapez un fait ou 'retour' pour quitter.
Fait reconnu : HeureCrime(14)

--- Mode : Ajout de faits ---
Tapez un fait ou 'retour' pour quitter.
Fait reconnu : Arme_Piece(Couteau,Phare)

--- Mode : Ajout de faits ---
Tapez un fait ou 'retour' pour quitter.
Fait reconnu : Personne_Piece(Docteur_Brown,Phare)

--- Mode : Ajout de faits ---
Tapez un fait ou 'retour' pour quitter.
Fait reconnu : EstMort(Gouverneur_White)

--- Mode : Ajout de faits ---
Tapez un fait ou 'retour' pour quitter.
Fait reconnu : Personne_Piece_Heure(Docteur_Brown,Phare,14)

--- Mode : Ajout de faits ---
Tapez un fait ou 'retour' pour quitter.
Aucun fichier de grammaire trou

## | Scenario de base (à copier-coller)

In [9]:
# Gouverneur_White est morte
# Gouverneur_White est dans le phare
# Gouverneur_White a des marques au cou
# Docteur_Brown est dans le phare
# Gouverneur_White est morte à 14h
# Docteur_Brown était dans le phare à 14h

Enfin, après avoir fourni tous les faits (scénario), nous demandons au moteur d’inférence de tirer ses conclusions.

In [15]:
# Conclusions
print("Pièce du crime : ", agent.get_crime_room())
print("Arme du crime : ", agent.get_crime_weapon())
print("Personne victime : ", agent.get_victim())
print("Heure du crime : ", agent.get_crime_hour())
print("Meurtrier : ", agent.get_suspect())
print("Personnes innocentes : ", agent.get_innocent())

Pièce du crime :  False
Arme du crime :  False
Personne victime :  Gouverneur_White
Heure du crime :  14
Meurtrier :  False
Personnes innocentes :  [Gouverneur_White]


## 3. INTERACTIONS TEXTE EN FRANÇAIS

In [18]:
import os
import sys
from nltk.parse import load_parser

GRAMMAR_DIR = os.path.join(os.getcwd(), "grammars")

def choisir_grammaire(phrase):
    phrase = phrase.lower()
    if "mort à" in phrase:
        return "personne_morte_heure.fcfg"
    elif "mort" in phrase:
        return "personne_morte.fcfg"
    elif "marque" in phrase:
        return "personne_marque.fcfg"
    elif any(arme in phrase for arme in ["corde", "couteau", "fusil", "clé", "cle", "matraque", "poison"]):
        return "arme_piece.fcfg"
    elif "était dans" in phrase or "est dans" in phrase or "se trouve" in phrase:
        return "personne_piece_heure.fcfg"
    else:
        return None

def lire_fait_francais(agent):
    phrase = input("→ Saisis une phrase (ex: Capitaine_Gray était dans le phare à 12h) :\n> ").strip()
    grammar_file = choisir_grammaire(phrase)
    if not grammar_file:
        print("Aucun fichier de grammaire trouvé pour cette phrase.")
        return

    grammar_path = os.path.join(GRAMMAR_DIR, grammar_file)
    tokens = phrase.split()

    try:
        parser = load_parser(f'file:{grammar_path}', trace=0)
        trees = list(parser.parse(tokens))
        sems = [tree.label()['SEM'] for tree in trees if 'SEM' in tree.label()]
        if sems:
            for sem in sems:
                print(f"Fait reconnu : {sem}")
                agent.add_clause(str(sem))
        else:
            print("Aucune sémantique détectée pour cette phrase.")
    except Exception as e:
        print(f"Erreur d'analyse avec {grammar_file} : {e}")


## 5 | AFFICHAGE DES CONCLUSIONS

In [20]:
# Fonction générique d'affichage de résultats logiques
def afficher_resultats(titre, requete_expr, *cles):
    print(f"\n{titre} :")
    resultats = list(agent.crime_kb.ask_generator(expr(requete_expr)))
    if not resultats:
        print("  Aucun résultat trouvé.")
        return

    deja_vu = set()
    for res in resultats:
        valeurs = tuple(str(res.get(expr(cle), "inconnu")) for cle in cles)
        if valeurs not in deja_vu:
            deja_vu.add(valeurs)
            print("  - " + " ".join(valeurs))

# Appels aux requêtes d'inférence
afficher_resultats("Possession d'arme", "Possede(x, Corde)", "x")
afficher_resultats("Suspects potentiels", "Suspect(x)", "x")
afficher_resultats("Arme du crime", "ArmeCrime(a)", "a")
afficher_resultats("Suspects avec arme du crime", "SuspectArme(x, a)", "x", "a")
afficher_resultats("Coupable", "Coupable(x)", "x")



Possession d'arme :
  - Docteur_Brown

Suspects potentiels :
  Aucun résultat trouvé.

Arme du crime :
  Aucun résultat trouvé.

Suspects avec arme du crime :
  Aucun résultat trouvé.

Coupable :
  Aucun résultat trouvé.
